In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from retry_requests import retry
import pickleshare

In [33]:
def call_open_meteo_averages(selected_station):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
    retry_strategy = Retry(total=5, backoff_factor=0.2)
    adapter = HTTPAdapter(max_retries=retry_strategy)
    cache_session.mount('http://', adapter)
    cache_session.mount('https://', adapter)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    # Initialize an empty list to store daily data
    daily_data = []

    # Define the start and end dates for the loop
    start_date = pd.Timestamp("1999-12-01")
    end_date = pd.Timestamp("1999-12-31")

    # Iterate through each day of December
    while start_date <= end_date:
        # Define the parameters for the API call
        params = {
            "latitude": selected_station["lat"],
            "longitude": selected_station["lon"],
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": start_date.strftime("%Y-%m-%d"),  # Use the same start and end date for daily data
            "daily": "wind_speed_10m_max",
            "timezone": "Europe/Berlin",
        }

        # Make the API call
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)

        for response in responses:
            # Process daily data. The order of variables needs to be the same as requested.
            daily = response.Daily()
            daily_wind_speed_10m_max = daily.Variables(0).ValuesAsNumpy()

            # Calculate the average wind speed for the day
            avg_wind_speed = daily_wind_speed_10m_max.mean()

            # Append the average wind speed for the day to daily_data
            daily_data.append({
                "date": start_date.strftime('%Y-%m-%d'),  # Format date to include year, month, and day
                "wind_speed_10m_max": avg_wind_speed,  # Use the average wind speed
            })

        # Move to the next day
        start_date += pd.Timedelta(days=1)

    # Create a DataFrame from daily data
    daily_dataframe = pd.DataFrame(daily_data)
    
    # Calculate daily averages
    daily_averages = daily_dataframe.groupby('date').mean()

    return daily_averages


In [34]:
Deutschland_Berlin={'lat': 52.5200, 'lon': 13.4050}
Deutschland_Berlin_wmean = call_open_meteo_averages(Deutschland_Berlin)
Deutschland_Berlin_wmean

,wind_speed_10m_max
date,
1999-12-01,31.624725
1999-12-02,32.491867
1999-12-03,38.672779
1999-12-04,39.477081
1999-12-05,24.944130
1999-12-06,30.599998
1999-12-07,28.772987
1999-12-08,30.821548
1999-12-09,21.749483
